## 1. Data of Interests

In the proposal, most projects only focus on a subset of data. Please state the subset of data to explore again here.

Our questions we're trying to answer is "Which cuisines are most positively received in certain region (West, Midwest, Northeast, and South) of the continental United States?” We are going to take the yelp data set and create sub sets or entries based off of region. The states will be divided up by region 

## 2. Data Preprocessing

In the proposal, most projects only focus on a subset of data. Please state the subset of data to explore again here.

## 3. EDA

Describe in detail what EDA and Statistical Testing are performed. You should perform at least three meaningful plots/testings. Please also summarize the insights from EDA.